# Korišćenje neuronskih mreža u regresionim zadacima

Ovaj primer će demonstrirati korišćenje Keras biblioteke u zadacima regresije. Na osnovu već poznatog skupa podataka Boston Housing koji sadrži informacije o nekretninama, potrebno je napraviti neuronski model koji će predvideti cenu nekretnine. Uz ovo, naučićemo i kako možemo sačuvati naučene modele i kasnije ih upotrebiti. 

Pre nego li počnemo sa radom, uključićemo sve potrebne biblioteke.

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Activation
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import boston_housing

In [2]:
import numpy as np
np.random.seed(7)

In [3]:
from matplotlib import pyplot as plt

In [4]:
from sklearn import preprocessing

In [5]:
from sklearn import preprocessing
from sklearn import model_selection

### Korak 1: Priprema skupa podataka

Prvo ćemo učitati skup podataka. Iskoristićemo *Boston housing* skup podataka Keras biblioteke dostupan kroz `dataset` modul koji objedinjuje nekoliko poznatih skupova podataka. Generisani skupovi za treniranje i testiranje su veću u formi `numpy` nizova.

### Korak 2: Kreiranje mreže

Mreža koju ćemo napraviti će biti mreža sa propagacijom unapred. Imaće ulazni sloj, jedan skriveni sloj sa 100 neurona i izlazni sloj. Broj neurona ulaznog sloja odgovara broju atributa skupa podataka (13). Broj neurona izlaznog sloja je 1 jer mreža treba da predvidi cenu nekretnine. Za aktivacionu funkciju prvog sloja odabraćemo `ReLu`, dok ćemo za drugi, izlazni sloj, ostaviti podrazumevanu linearnu aktivaciju `a(x)=x` koja je standardni izbor u slučajevima regresionih zadataka. Druge aktivacione funkcije bi ograničavale opsege koje mreža može da nauči što je u suprotnosti sa očekivanjima regresionih zadataka. 

### Korak 3: Treniranje mreže

Alternativni način zadavanja optimizatora uz finija podešavanja parametara se može postići uključivanjem `optimizer` paketa, a funkcija gubitka uključivanjem `losses` paketa. Zahvaljujući ovim paketima umesto inicijalizacije niskama, potrebne parametre možemo inicijalizovati instancama odgovarajućih klasa. Tako ćemo prilikom treniranja mreže odabrati srednjekvadratnu grešku (engl. mean squared error) predstavljenu klasom `MeanSquaredError` kao funkciju greške, a srednju apsolutnu grešku predstavljenu klasom `MeanAbsoluteError` kao funkciju čiju ćemo vrednost pratiti u toku obučavanja. Kao optimizator ćemo odabrati Adam. On je predstavljen klasom `Adam`. 

Sada ćemo prilikom treniranja mreže, jedan deo podataka iskoristiti za validaciju. Parametar `validation_split` omogućava da se u zadatoj srazmeri izdvoji deo skupa za treniranje koji će biti korišćen isključivo za validaciju, tj. za monitoring rada mreže. Na kraju svake epohe nad ovim skupom će biti izračunate i sačuvane metrike zadate na nivou modela. Alternativa ovom pristupu bi bilo zadavanje eksplicitno pripremljenog validacionog skupa u formi `(X_validation, y_validation)` preko parametra `validation_data`.

Mrežu ćemo trenirati u 100 epoha sa paketićima veličine 32 instance. Ovoga puta ćemo postaviti i `verbose` parametar funkcije na vrednost 1 kako bismo mogli da pratimo ispise u toku izvršavanja funkcije.

### Korak 4: Evaluacija mreže

Keras biblioteka omogućava i definisanje funkcija sa povratnim pozivom (engl. callback functions) kojima se mogu pratiti željene vrednosti na nivou obrade pojedinačnih paketića ili celih epoha. Prilikom opredeljivanja za praćenje ovih metrika treba uzeti u obzir to da li su globalnog ili lokalnog karaktera i da li ih ima smisla uporosečavati u krajnju vrednost ili ne. Na primer, srednjekavadratna greška se može pratiti na nivou paketa i na kraju uprosečiti, dok to ne važi za koeficijent determinacije. 

Sledeći primer ilustruje definisanje omotača koji koristi predefinisane funkcije `on_train_begin` i `on_batch_end`. Slično njima, postoje i funkcije `on_train_end` i `on_batch_begin`, kao i funkcije koje se tiču praćenja epoha `on_epoch_begin` i `on_epoch_end`. Gde je primenljivo, ove funkcije imaju i pridruženi paketić sa podacima (parametar `batch`). Parametar `logs` koji se pominje je trenutno fiktivnog karaktera i uvek ga treba postaviti na {}.

In [25]:
from tensorflow.keras import callbacks

Funkcije sa povratnim pozivom se mogu iskoristiti i za nešto drugačije svrhe. Na primer, često je, ukoliko nema progresa prilikom obučavanja mreže, praktično prevremeno prekinuti obučavanje. Za te svrhe se može koristiti funkcija `EarlyStopping`. Ova funkcija dozvoljava da se parametrom `monitor` naznači metrika koju treba pratiti kao i broj epoha koji treba tolerisati za slučaj da nema progresa - za to se upotrebljava parametar `patience`. Sama funkcija se navodi prilikom poziva metode `fit` u okviru `callbacks` niza. 

In [32]:
from tensorflow.keras.callbacks import EarlyStopping

### Korak 5: Čuvanje modela

Zajednica koja se bavi mašinskim učenjem ulaže napore da standardizuje formate za čuvanje mreža. Jedan od njih je i [ONNX](https://onnx.ai/) za koji Keras biblioteka nudi [podršku](https://github.com/onnx/onnx-docker/blob/master/onnx-ecosystem/converter_scripts/keras_onnx.ipynb). O ovim temam će svakako biti više reči i u nastavku kursa. 
<img src='assets/onnx.png'>

Ukoliko nam je potreban uvid u samu arhitekturu mreže, možemo generisati i grafički prikaz korišćenjem pomoćnih funkcionalnosti biblioteke keras. Za ova generisanja nam je potrebna i bibliteka `graphviz`.

In [45]:
from tensorflow.keras import utils

In [ ]:
utils.plot_model(model, to_file='model_keras_regresija.png', show_shapes=True)

Na grafiku su prikazana podrazumevana imena slojeva. Adekvatnija imena se mogu zadati svojstvom `name`. 

Vizuelizacije mreže će biti moguće i kroz alat [TensorBoard](https://www.tensorflow.org/tensorboard) koji ćemo, takođe, upoznati na nekom od narednih časova.